In [1]:
import os
import warnings
import intake
import xarray as xr 
import numpy as np
import copy

warnings.filterwarnings('ignore')

In [2]:
dataframe = intake.open_esm_datastore("https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json")

In [18]:
dataframe

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


## Historical dataset ("gr")
Search for near surface air temperature and precipitation:

In [22]:
hist_tas=dataframe.search(experiment_id = 'historical',  # select historical simulations
                 variable_id = ['tas'],             # tas = near surface air temperature
                 table_id ='Amon',                  # Amon = monthly data
                 grid_label ='gr',                  # gr =grid info
                 member_id = 'r1i1p1f1')            # member_id: realisation          
hist_tas_ds = hist_tas.to_dataset_dict()
hist_pr=dataframe.search(experiment_id='historical',  # select historical simulations
                 variable_id=['pr'],             # pr=precipitation
                 table_id='Amon',                # Amon = monthly data
                 grid_label='gr',
                 member_id = 'r1i1p1f1') 
#hist_pr_ds = hist_pr.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


ESMDataSourceError: Failed to load dataset with key='CMIP.EC-Earth-Consortium.EC-Earth3-CC.historical.Amon.gr'
                 You can use `cat['CMIP.EC-Earth-Consortium.EC-Earth3-CC.historical.Amon.gr'].df` to inspect the assets/files for this key.
                 

In [6]:
hist_tas, hist_pr

(<pangeo-cmip6 catalog with 14 dataset(s) from 14 asset(s)>,
 <pangeo-cmip6 catalog with 13 dataset(s) from 13 asset(s)>)

Select only dataset which contain both (tas & pr data)

In [4]:
key_list = list(hist_tas.keys() & hist_pr.keys())
source_ids = np.sort([key.split('.')[2] for key in key_list])
source_ids

array(['E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'EC-Earth3',
       'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg',
       'EC-Earth3-Veg-LR', 'FGOALS-f3-L', 'IPSL-CM5A2-INCA',
       'IPSL-CM6A-LR', 'IPSL-CM6A-LR-INCA', 'KACE-1-0-G'], dtype='<U17')

## SSP245 Scenarios
We check for which gcm a corresponding ssp245 scenario exists:

In [5]:
ssp_tas=dataframe.search(source_id = source_ids,
                 experiment_id='ssp245',# select historical simulations
                 variable_id = ['tas'],             # tas = near surface air temperature
                 table_id ='Amon',                  # Amon = monthly data
                 grid_label ='gr',                  # gr =grid info
                 member_id = 'r1i1p1f1')            # member_id: realisation  
ssp_tas_ds = ssp_tas.to_dataset_dict()
ssp_pr=dataframe.search(source_id=source_ids,
                    experiment_id='ssp245',  # select historical simulations
                     variable_id=['pr'],             # pr=precipitation
                     table_id='Amon',                # Amon = monthly data
                     grid_label='gr',
                     member_id = 'r1i1p1f1') 
ssp_pr_ds = ssp_pr.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Check, which source_ids are identical with those from the historical dataset

In [6]:
merge_id = ['.'+id.split('.')[2]+'.' for id in list(ssp_tas.keys() & ssp_pr.keys())]
# drop ".E3SM-1-1." as it starts in 2020 only
merge_id.remove(".E3SM-1-1.")
merge_id

['.KACE-1-0-G.',
 '.EC-Earth3-CC.',
 '.EC-Earth3-Veg-LR.',
 '.FGOALS-f3-L.',
 '.EC-Earth3.',
 '.EC-Earth3-Veg.',
 '.IPSL-CM6A-LR.']

## Merge and write the files

In [95]:
for key in list(hist_tas.keys()&hist_pr.keys())[:1]:
    #check if ssp245 scenario exists
    source_id = '.'+key.split('.')[2]+'.'
    if source_id in merge_id:
        ssp_key = [key for key in list(ssp_tas.keys()) if source_id in key][0]
        # merge years from ssp245 scenario with historical data
        hist_tas_ds[key]=xr.merge([hist_tas_ds[key],ssp_tas_ds[ssp_key].sel(time=slice("2015", "2034"))])
        hist_pr_ds[key]=xr.merge([hist_pr_ds[key],ssp_pr_ds[ssp_key].sel(time=slice("2015", "2034"))])
    hist_tas_ds[key].sel(member_id='r1i1p1f1').to_netcdf(os.path.join('cmip6','tas',key+'_tas.nc'))
    hist_pr_ds[key].sel(member_id='r1i1p1f1').to_netcdf(os.path.join('cmip6','pr',key+'_pr.nc'))

In [37]:
for key in list(ssp_tas.keys()&ssp_pr.keys()):
    #check if ssp245 scenario exists
    source_id = '.'+key.split('.')[2]+'.'
    if source_id in merge_id:
        hist_key = [key for key in list(hist_tas.keys()) if source_id in key][0]
        print(source_id)

.FGOALS-f3-L.
.IPSL-CM6A-LR.
.KACE-1-0-G.
.EC-Earth3-CC.
.EC-Earth3-Veg.
.EC-Earth3.
.EC-Earth3-Veg-LR.


In [ ]:
for key in list(ssp_tas.keys()&ssp_pr.keys()):
    #check if ssp245 scenario exists
    source_id = '.'+key.split('.')[2]+'.'
    if source_id in merge_id:
        hist_key = [key for key in list(hist_tas.keys()) if source_id in key][0]
        print(source_id)
        # merge years from ssp245 scenario with historical data
        hist_tas_ds[hist_key]=xr.merge([hist_tas_ds[hist_key],ssp_tas_ds[key].sel(time=slice("2015", "2034"))])
        hist_pr_ds[hist_key]=xr.merge([hist_pr_ds[hist_key],ssp_pr_ds[key].sel(time=slice("2015", "2034"))])
    print('write netcdf')
    hist_tas_ds[hist_key].sel(member_id='r1i1p1f1').to_netcdf(os.path.join('cmip6','tas',hist_key+'_tas.nc'))
    hist_pr_ds[hist_key].sel(member_id='r1i1p1f1').to_netcdf(os.path.join('cmip6','pr',hist_key+'_pr.nc'))

In [50]:
ssp_tas.keys()&ssp_pr.keys(), merge_id

({'ScenarioMIP.CAS.FGOALS-f3-L.ssp245.Amon.gr',
  'ScenarioMIP.E3SM-Project.E3SM-1-1.ssp245.Amon.gr',
  'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp245.Amon.gr',
  'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.ssp245.Amon.gr',
  'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.Amon.gr',
  'ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.Amon.gr',
  'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp245.Amon.gr',
  'ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp245.Amon.gr'},
 ['.KACE-1-0-G.',
  '.EC-Earth3-CC.',
  '.EC-Earth3-Veg-LR.',
  '.FGOALS-f3-L.',
  '.EC-Earth3.',
  '.EC-Earth3-Veg.',
  '.IPSL-CM6A-LR.'])

In [54]:
for key in list(['ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp245.Amon.gr']):
    #check if ssp245 scenario exists
    source_id = '.'+key.split('.')[2]+'.'
    if source_id in merge_id:
        hist_key = [key for key in list(hist_tas.keys()) if source_id in key][0]
        # get historical dataset
        ds_pr = copy.deepcopy(hist_pr_ds[hist_key])
        ds_tas = copy.deepcopy(hist_tas_ds[hist_key])
        
        # merge historical dataset with ssp data
        ds_pr = xr.merge([ds_pr,ssp_pr_ds[key].sel(time=slice("2015", "2034"))])
        ds_tas = xr.merge([ds_tas,ssp_tas_ds[key].sel(time=slice("2015", "2034"))])
        
        # delete if file exists and rewrite
        p_pr = os.path.join('cmip6','pr',hist_key+'_pr.nc')
        p_tas = os.path.join('cmip6','tas',hist_key+'_tas.nc')
        if os.path.isfile(p_pr):
            os.remove(p_pr)
        if os.path.isfile(p_tas):
            os.remove(p_tas)
        ds_pr.sel(member_id='r1i1p1f1').to_netcdf(p_pr)
        ds_tas.sel(member_id='r1i1p1f1').to_netcdf(p_tas)

In [56]:
ds_pr.sel(member_id='r1i1p1f1')
#ds_tas.sel(member_id='r1i1p1f1')

<xarray.Dataset>
Dimensions:      (time: 2220, lat: 143, lon: 144, axis_nbounds: 2)
Coordinates:
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2034-12-16T12:...
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(2220, 2), meta=np.ndarray>
    member_id    <U8 'r1i1p1f1'
Dimensions without coordinates: axis_nbounds
Data variables:
    pr           (time, lat, lon) float32 dask.array<chunksize=(2220, 143, 144), meta=np.ndarray>
Attributes: (12/56)
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    Conventions:             CF-1.7 CMIP-6.2
    EXPID:                   historical
    NCO:                     "4.6.0"
    activity_id:             CMIP
    branch_method:           standard
    ...                      ...
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:     hdl:21.14100/f63dbc41-c629-41a4-8646-c3423a1f1379
    version_id:              v20180803
    intake_esm_varname:      ['pr']
    intake_esm_dataset_key:  CMIP.IPSL.IPSL-CM6A-LR.historical.Amon.gr

In [101]:
list(ssp_tas.keys()&ssp_pr.keys())[1:2]

['ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.Amon.gr']

In [21]:
ssp_tas_ds[key].sel(time=slice("2015", "2035"))

<xarray.Dataset>
Dimensions:      (lat: 143, lon: 144, member_id: 1, time: 252, axis_nbounds: 2)
Coordinates:
    height       float64 ...
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time         (time) datetime64[ns] 2015-01-16T12:00:00 ... 2035-12-16T12:...
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(252, 2), meta=np.ndarray>
  * member_id    (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: axis_nbounds
Data variables:
    tas          (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 252, 143, 144), meta=np.ndarray>
Attributes: (12/56)
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    Conventions:             CF-1.7 CMIP-6.2
    EXPID:                   ssp245
    activity_id:             ScenarioMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    ...                      ...
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:     hdl:21.14100/1af86fe0-e971-4c29-a3bf-7ecd31542ef9
    version_id:              v20190119
    intake_esm_varname:      ['tas']
    intake_esm_dataset_key:  ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp245.Amon.gr

In [26]:
hist_tas_ds[hist_key]

<xarray.Dataset>
Dimensions:      (lat: 143, lon: 144, member_id: 1, time: 1980, axis_nbounds: 2)
Coordinates:
    height       float64 ...
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:...
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * member_id    (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: axis_nbounds
Data variables:
    tas          (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 600, 143, 144), meta=np.ndarray>
Attributes: (12/56)
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    Conventions:             CF-1.7 CMIP-6.2
    EXPID:                   historical
    NCO:                     "4.6.0"
    activity_id:             CMIP
    branch_method:           standard
    ...                      ...
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:     hdl:21.14100/285a3a27-0287-4e3b-8232-69ddc89cebef
    version_id:              v20180803
    intake_esm_varname:      ['tas']
    intake_esm_dataset_key:  CMIP.IPSL.IPSL-CM6A-LR.historical.Amon.gr

In [27]:
xr.merge([hist_tas_ds[hist_key],ssp_tas_ds[key].sel(time=slice("2015", "2035"))])

<xarray.Dataset>
Dimensions:      (time: 2232, lat: 143, lon: 144, member_id: 1, axis_nbounds: 2)
Coordinates:
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2035-12-16T12:...
    height       float64 2.0
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(2232, 2), meta=np.ndarray>
  * member_id    (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: axis_nbounds
Data variables:
    tas          (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 2232, 143, 144), meta=np.ndarray>
Attributes: (12/56)
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    Conventions:             CF-1.7 CMIP-6.2
    EXPID:                   historical
    NCO:                     "4.6.0"
    activity_id:             CMIP
    branch_method:           standard
    ...                      ...
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:     hdl:21.14100/285a3a27-0287-4e3b-8232-69ddc89cebef
    version_id:              v20180803
    intake_esm_varname:      ['tas']
    intake_esm_dataset_key:  CMIP.IPSL.IPSL-CM6A-LR.historical.Amon.gr

In [90]:
for key in list(hist_tas_ds.keys())[:2]:
    print(key,hist_tas_ds[key].time.attrs)

CMIP.E3SM-Project.E3SM-1-1-ECA.historical.Amon.gr {'axis': 'T', 'bounds': 'time_bnds', 'long_name': 'time', 'standard_name': 'time'}
CMIP.IPSL.IPSL-CM5A2-INCA.historical.Amon.gr {'axis': 'T', 'bounds': 'time_bounds', 'long_name': 'Time axis', 'standard_name': 'time', 'time_origin': '1850-01-01 00:00:00'}


## different grid label "gn"

get historical dataset with "gn" grid

In [24]:
hist_tas=dataframe.search(experiment_id = 'historical',  # select historical simulations
                 variable_id = ['tas'],             # tas = near surface air temperature
                 table_id ='Amon',                  # Amon = monthly data
                 grid_label ='gn',                  # gr =grid info
                 member_id = 'r1i1p1f1')            # member_id: realisation          
hist_tas_ds = hist_tas.to_dataset_dict()
hist_pr=dataframe.search(experiment_id='historical',  # select historical simulations
                 variable_id=['pr'],             # pr=precipitation
                 table_id='Amon',                # Amon = monthly data
                 grid_label='gn',
                 member_id = 'r1i1p1f1') 
hist_pr_ds = hist_pr.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


ESMDataSourceError: Failed to load dataset with key='CMIP.UA.MCM-UA-1-0.historical.Amon.gn'
                 You can use `cat['CMIP.UA.MCM-UA-1-0.historical.Amon.gn'].df` to inspect the assets/files for this key.
                 

In [7]:
len(hist_tas.keys() & hist_pr.keys())

36

In [9]:
key_list = list(hist_tas.keys() & hist_pr.keys())
source_ids = np.sort([key.split('.')[2] for key in key_list])
source_ids

array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR',
       'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2',
       'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CMCC-CM2-HR4',
       'CMCC-CM2-SR5', 'CMCC-ESM2', 'CanESM5', 'FGOALS-g3', 'FIO-ESM-2-0',
       'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-H',
       'ICON-ESM-LR', 'IITM-ESM', 'MCM-UA-1-0', 'MIROC6',
       'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
       'NESM3', 'NorCPM1', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON',
       'TaiESM1'], dtype='<U15')

check for corresponding SSP245 scenario

In [10]:
ssp_tas=dataframe.search(source_id = source_ids,
                 experiment_id='ssp245',# select historical simulations
                 variable_id = ['tas'],             # tas = near surface air temperature
                 table_id ='Amon',                  # Amon = monthly data
                 grid_label ='gn',                  # gr =grid info
                 member_id = 'r1i1p1f1')            # member_id: realisation  
ssp_tas_ds = ssp_tas.to_dataset_dict()
ssp_pr=dataframe.search(source_id=source_ids,
                    experiment_id='ssp245',  # select historical simulations
                     variable_id=['pr'],             # pr=precipitation
                     table_id='Amon',                # Amon = monthly data
                     grid_label='gn',
                     member_id = 'r1i1p1f1') 
ssp_pr_ds = ssp_pr.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
█
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [11]:
merge_id = ['.'+id.split('.')[2]+'.' for id in list(ssp_tas.keys() & ssp_pr.keys())]
merge_id

['.FGOALS-g3.',
 '.MPI-ESM1-2-HR.',
 '.CESM2-WACCM.',
 '.FIO-ESM-2-0.',
 '.NESM3.',
 '.IITM-ESM.',
 '.CanESM5.',
 '.CMCC-ESM2.',
 '.MPI-ESM1-2-LR.',
 '.CMCC-CM2-SR5.',
 '.NorESM2-LM.',
 '.CAS-ESM2-0.',
 '.MIROC6.',
 '.BCC-CSM2-MR.',
 '.AWI-CM-1-1-MR.',
 '.ACCESS-CM2.',
 '.TaiESM1.',
 '.MRI-ESM2-0.',
 '.NorESM2-MM.',
 '.CAMS-CSM1-0.']

In [52]:
for key in list(hist_tas.keys() & hist_pr.keys()):
    
    ds_pr=copy.deepcopy(hist_pr_ds[key])
    ds_tas=copy.deepcopy(hist_tas_ds[key])
    
    try:
        #check if ssp245 scenario exists
        source_id = '.'+key.split('.')[2]+'.'
        if source_id in merge_id:
            # get from ssp set
            ssp_key = [key for key in list(ssp_tas.keys()) if source_id in key][0]

            ds_pr = xr.merge([ds_pr, ssp_pr_ds[ssp_key].sel(time=slice("2015","2034"))])
            ds_tas = xr.merge([ds_tas, ssp_tas_ds[ssp_key].sel(time=slice("2015","2034"))])
    except:
        print('merge failed at',key)
        ds_pr=copy.deepcopy(hist_pr_ds[key])
        ds_tas=copy.deepcopy(hist_tas_ds[key]) 

    # delete if file exists and rewrite
    p_pr = os.path.join('cmip6_gn','pr',key+'_pr.nc')
    p_tas = os.path.join('cmip6_gn','tas',key+'_tas.nc')  
    try:
        ds_pr.sel(member_id='r1i1p1f1').to_netcdf(p_pr)
        ds_tas.sel(member_id='r1i1p1f1').to_netcdf(p_tas)
        print('save',key)
    except:
        print('saving failed',key)
    

save CMIP.BCC.BCC-ESM1.historical.Amon.gn


In [156]:
i=0
for key in list(hist_tas.keys() & hist_pr.keys()):
    p_pr = os.path.join('cmip6_gn','pr',key+'_pr.nc')
    p_tas = os.path.join('cmip6_gn','tas',key+'_tas.nc')  
    if os.path.isfile(p_pr)&os.path.isfile(p_tas):
        tas_ds = xr.open_dataset(p_tas)
        pr_dr= xr.open_dataset(p_pr)
        try:
            print( tas_ds.time.values[-1], key)
        except:
            print(key)

2034-12-16 12:00:00 CMIP.CMCC.CMCC-CM2-SR5.historical.Amon.gn
2034-12-16 12:00:00 CMIP.AS-RCEC.TaiESM1.historical.Amon.gn
2014-12-15 12:00:00 CMIP.NCAR.CESM2-WACCM-FV2.historical.Amon.gn
2014-12-16 12:00:00 CMIP.NASA-GISS.GISS-E2-1-G.historical.Amon.gn
2014-12-16T12:00:00.000000000 CMIP.AWI.AWI-CM-1-1-MR.historical.Amon.gn
2014-12-16 12:00:00 CMIP.NASA-GISS.GISS-E2-2-H.historical.Amon.gn
2014-12-16T12:00:00.000000000 CMIP.AWI.AWI-ESM-1-1-LR.historical.Amon.gn
2034-12-16T12:00:00.000000000 CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn
2034-12-16 12:00:00 CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn
2034-12-16 12:00:00 CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn
2014-12-16T12:00:00.000000000 CMIP.CSIRO.ACCESS-ESM1-5.historical.Amon.gn
2034-12-16 12:00:00 CMIP.FIO-QLNM.FIO-ESM-2-0.historical.Amon.gn
2014-12-16T12:00:00.000000000 CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.Amon.gn
2034-12-16T12:00:00.000000000 CMIP.MRI.MRI-ESM2-0.historical.Amon.gn
2034-12-16 12:00:00 CMIP.CAS.CAS-ESM2-0.h

In [173]:
key='CMIP.UA.MCM-UA-1-0.historical.Amon.gn'
p_pr = os.path.join('cmip6_gn','pr',key+'_pr.nc')
p_tas = os.path.join('cmip6_gn','tas',key+'_tas.nc')  
tas_ds = xr.open_dataset(p_tas)
pr_ds = xr.open_dataset(p_pr)
tas_ds.dims, pr_ds.dims

(Frozen({}), Frozen({}))

In [167]:
key ='CMIP.UA.MCM-UA-1-0.historical.Amon.gn'
#key='CMIP.BCC.BCC-ESM1.historical.Amon.gn'
p_pr = os.path.join('cmip6_gn','pr',key+'_pr.nc')
p_tas = os.path.join('cmip6_gn','tas',key+'_tas.nc')  
hist_tas_ds[key].sel(member_id='r1i1p1f1').drop(['height','time_bnds']).to_netcdf(p_tas)
hist_pr_ds[key].sel(member_id='r1i1p1f1').drop(['time_bnds']).to_netcdf(p_pr)

In [174]:
hist_pr_ds[key]

<xarray.Dataset>
Dimensions:    (latitude: 80, longitude: 96, bnds: 2, member_id: 1, time: 1980)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    lat_bnds   (latitude, bnds) float64 dask.array<chunksize=(80, 2), meta=np.ndarray>
  * latitude   (latitude) float64 -88.29 -86.07 -83.84 ... 83.84 86.07 88.29
    lon_bnds   (longitude, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * longitude  (longitude) float64 0.0 3.75 7.5 11.25 ... 348.8 352.5 356.2
  * time       (time) object 1850-01-17 00:00:00 ... 2014-12-17 00:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    areacella  (latitude, longitude) float64 dask.array<chunksize=(80, 96), meta=np.ndarray>
    pr         (member_id, time, latitude, longitude) float32 dask.array<chunksize=(1, 990, 80, 96), meta=np.ndarray>
Attributes: (12/51)
    Conventions:             CF-1.7 CMIP-6.2 
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    1533000.0
    branch_time_in_parent:   0.0
    comment:                 Blank                                 
    ...                      ...
    variant_info:            1850 equiv CO2 and aerosol forcing    
    variant_label:           r1i1p1f1
    netcdf_tracking_ids:     hdl:21.14100/88516eba-408b-41b6-b2fb-574be9816556
    version_id:              v20191017
    intake_esm_varname:      ['pr']
    intake_esm_dataset_key:  CMIP.UA.MCM-UA-1-0.historical.Amon.gn

In [23]:
## find areacello

In [ ]:
hist_tas=dataframe.search(experiment_id = 'historical',  # select historical simulations
                 variable_id = ['tas'],             # tas = near surface air temperature
                 table_id ='Amon',                  # Amon = monthly data
                 grid_label ='gr',                  # gr =grid info
                 member_id = 'r1i1p1f1')            # member_id: realisation          
hist_tas_ds = hist_tas.to_dataset_dict()